<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Project-Check-in" data-toc-modified-id="Final-Project-Check-in-1">Final Project Check-in</a></span></li><li><span><a href="#Group-Name" data-toc-modified-id="Group-Name-2">Group Name</a></span></li><li><span><a href="#Student-Names" data-toc-modified-id="Student-Names-3">Student Names</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4">Load Data</a></span></li><li><span><a href="#Fit-scikit-learn-model" data-toc-modified-id="Fit-scikit-learn-model-5">Fit scikit-learn model</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-6">Evaluation Metric</a></span></li></ul></div>

Final Project Check-in
------

Group Name: SAL
-----

Student Names
----

1. Lisa Chua
2. Alaa A Latif
3. Shane Buchanan 

Load Data
-----

In [8]:
reset -fs

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, SMOTENC

In [10]:
def make_pipeline(classifier=None):
    "Create a single pipeline that processes the data and then fits the regressor." 
    
    # YOUR CODE HERE
    numeric_features     = ['age', 'campaign', 'pdays', 'previous']
    categorical_features = ['job', 'education', 'default', 'housing',
                            'loan', 'campaign', 'poutcome', 'marital', 'contact', 'month']
    
    categorical_transformer = Pipeline(steps=[
    ('impute', impute.SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])
    
    numeric_transformer = Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='median')),
    ('scaler', preprocessing.StandardScaler())])
    
    
    
    preprocessor = compose.ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', classifier)])
    return pipeline

In [11]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('~/data/bank-additional/bank-additional-full.csv', delimiter=';')

In [12]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [13]:
t = (df.dtypes == 'object')
trues = [i for i, x in enumerate(t) if x][:-1]
trues

[1, 2, 3, 4, 5, 6, 7, 8, 9, 14]

In [14]:
# Features data.
X = df.drop('y', axis=1)

In [15]:
y = df['y']

In [27]:
# Train-test-split on data.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [17]:
(y_train == 'yes').sum()

3734

### Rebalance the data set

In [32]:
def up_sample(X, y):
    rows_idxs_no = y == 'no'
    rows_idxs_yes = y == 'yes'
    df_yes = pd.concat((X.loc[rows_idxs_yes], y.loc[rows_idxs_yes]), axis = 1)
    df_no = pd.concat((X.loc[rows_idxs_no], y.loc[rows_idxs_no]), axis = 1)
    df_yes_upsampled = df_yes.sample(int(len(df_yes)*5), replace=True)
    df_upsampled = pd.concat([df_no, df_yes_upsampled], axis=0)
    return df_upsampled.drop('y', axis=1), df_upsampled['y']

# K Fold Cross Validation

In [19]:
kf = KFold(n_splits=5, shuffle=True)

In [20]:
# smt = SMOTENC(trues)
# X_train_up, y_train_up = smt.fit_sample(X_train, y_train)
# X_train_up = pd.DataFrame(X_train_up, columns=X.columns)
# y_train_up = pd.Series(y_train_up)
X_train_up = X_train
y_train_up = y_train
for train_index, test_index in kf.split(X_train_up):
    X_train_up1, X_val_up = X_train_up.iloc[train_index], X_train_up.iloc[test_index]
    y_train_up1, y_val_up = y_train_up.iloc[train_index], y_train_up.iloc[test_index]
    #clf = DecisionTreeClassifier()
    clf = LogisticRegression()
#     clf = RandomForestClassifier()
    pipeline = make_pipeline(clf)
    pipeline.fit(X_train_up1, y_train_up1)
    y_pred = pipeline.predict(X_val_up)
    print(f1_score(y_val_up, y_pred, average='weighted'))

/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8770408828200905


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8677549937108577


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.86704572913023


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8713935458277658


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8647655343746348


Fit scikit-learn model
----

In [21]:
# # Initialize dtree and fit.
# clf = DecisionTreeClassifier()
# pipeline = make_pipeline(clf)

## Predict

In [22]:
# # Train Estimator
pipeline.fit(X_train_up, y_train_up);

/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [23]:
y_pred = pipeline.predict(X_test)

In [24]:
confusion_matrix(y_test, y_pred)

array([[7240,   92],
       [ 733,  173]])

Evaluation Metric
----

In [25]:
# Use F1 score as metric since we have an unbalanced data set.
f1_score(y_test, y_pred, average='weighted')

0.874541832865125

In [26]:
accuracy_score(y_test, y_pred)

0.8998543335761107

## Hyperparameter Tuning - Logistic

In [28]:
# Train-test-split on data.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

### Exhaustive Search (Coarse)

In [35]:
models = {}
penalty = ['l1', 'l2', 'elasticnet', 'none']
class_wght = ['balanced', None]
solver = ['newton-cg', 'lbfgs', 'sag', 'saga', 'liblinear']
alphas = [0.1, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5]
tols = [0.01, 0.1, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5]
for c_wght in class_wght:
    for a in alphas:
        for t in tols:
            for p in penalty:
                if p == 'elasticnet':
                    clf = LogisticRegression(penalty=p, tol=t, C=a, class_weight=c_wght, solver='saga', l1_ratio=0.5)
                    pipeline = make_pipeline(clf)
                    pipeline.fit(X_train, y_train)
                    y_pred_trn = pipeline.predict(X_train)
                    trn_score = f1_score(y_train, y_pred_trn, average='weighted')
                    y_pred_val = pipeline.predict(X_val)
                    val_score = f1_score(y_val, y_pred_val, average='weighted')
                    print(f'pen: {p}, tol: {t}, C: {a}, class weight: {c_wght}, solver: {s}, train score: {trn_score:.2f}, val score: {val_score:.2f}')
                elif p == 'none':
                    for s in solver[:-1]:
                        clf = LogisticRegression(penalty=p, tol=t, C=a, class_weight=c_wght, solver=s)
                        pipeline = make_pipeline(clf)
                        pipeline.fit(X_train, y_train)
                        y_pred_trn = pipeline.predict(X_train)
                        trn_score = f1_score(y_train, y_pred_trn, average='weighted')
                        y_pred_val = pipeline.predict(X_val)
                        val_score = f1_score(y_val, y_pred_val, average='weighted')
                        print(f'pen: {p}, tol: {t}, C: {a}, class weight: {c_wght}, solver: {s}, train score: {trn_score:.2f}, val score: {val_score:.2f}')
                elif p == 'l1':
                    for s in solver[3:]:
                        clf = LogisticRegression(penalty=p, tol=t, C=a, class_weight=c_wght, solver=s)
                        pipeline = make_pipeline(clf)
                        pipeline.fit(X_train, y_train)
                        y_pred_trn = pipeline.predict(X_train)
                        trn_score = f1_score(y_train, y_pred_trn, average='weighted')
                        y_pred_val = pipeline.predict(X_val)
                        val_score = f1_score(y_val, y_pred_val, average='weighted')
                        print(f'pen: {p}, tol: {t}, C: {a}, class weight: {c_wght}, solver: {s}, train score: {trn_score:.2f}, val score: {val_score:.2f}')
                else:
                    for s in solver:
                        clf = LogisticRegression(penalty=p, tol=t, C=a, class_weight=c_wght, solver=s)
                        pipeline = make_pipeline(clf)
                        pipeline.fit(X_train, y_train)
                        y_pred_trn = pipeline.predict(X_train)
                        trn_score = f1_score(y_train, y_pred_trn, average='weighted')
                        y_pred_val = pipeline.predict(X_val)
                        val_score = f1_score(y_val, y_pred_val, average='weighted')
                        print(f'pen: {p}, tol: {t}, C: {a}, class weight: {c_wght}, solver: {s}, train score: {trn_score:.2f}, val score: {val_score:.2f}')

pen: l1, tol: 0.01, C: 0.1, class weight: balanced, solver: saga, train score: 0.83, val score: 0.83
pen: l1, tol: 0.01, C: 0.1, class weight: balanced, solver: liblinear, train score: 0.83, val score: 0.83
pen: l2, tol: 0.01, C: 0.1, class weight: balanced, solver: newton-cg, train score: 0.83, val score: 0.83


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


pen: l2, tol: 0.01, C: 0.1, class weight: balanced, solver: lbfgs, train score: 0.83, val score: 0.83
pen: l2, tol: 0.01, C: 0.1, class weight: balanced, solver: sag, train score: 0.83, val score: 0.83
pen: l2, tol: 0.01, C: 0.1, class weight: balanced, solver: saga, train score: 0.83, val score: 0.83
pen: l2, tol: 0.01, C: 0.1, class weight: balanced, solver: liblinear, train score: 0.83, val score: 0.83
pen: elasticnet, tol: 0.01, C: 0.1, class weight: balanced, solver: liblinear, train score: 0.83, val score: 0.83


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


pen: none, tol: 0.01, C: 0.1, class weight: balanced, solver: newton-cg, train score: 0.83, val score: 0.83


...

/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


pen: none, tol: 1.5, C: 1.5, class weight: balanced, solver: sag, train score: 0.68, val score: 0.70


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1510: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


pen: none, tol: 1.5, C: 1.5, class weight: balanced, solver: saga, train score: 0.79, val score: 0.79
pen: l1, tol: 0.01, C: 0.1, class weight: None, solver: saga, train score: 0.87, val score: 0.87


ValueError: class_weight must be dict, 'balanced', or None, got: 'None'

Top Logistic Models:
* pen: l1, tol: 0.01, C: 0.1, class weight: None, solver: saga, train score: 0.87, val score: 0.87
* pen: l2, tol: 1.25, C: 0.75, class weight: balanced, solver: sag, train score: 0.86, val score: 0.87
* pen: l2, tol: 1.5, C: 0.5, class weight: balanced, solver: sag, train score: 0.86, val score: 0.86
* pen: none, tol: 1.25, C: 0.25, class weight: balanced, solver: sag, train score: 0.86, val score: 0.87
* pen: l2, tol: 1.0, C: 0.25, class weight: balanced, solver: sag, train score: 0.86, val score: 0.86
* pen: l2, tol: 0.75, C: 0.25, class weight: balanced, solver: sag, train score: 0.86, val score: 0.86

### Exhaustive Search (Fine)

In [38]:
penalty = ['l1', 'l2']
class_wght = ['balanced', None]
solver = ['sag', 'saga']
alphas = [0.01, 0.025, 0.05, 0.075, 0.1, 0.70, 0.75, 0.8, 0.85, 0.9]
tols = [0.0025, 0.005, 0.0075, 0.001, 1.20, 1.25, 1.30, 1.35, 1.4, 1.45, 1.5]
for c_wght in class_wght:
    for a in alphas:
        for t in tols:
            for p in penalty:
                if p == 'l2':
                    
                    for s in solver:
                        clf = LogisticRegression(penalty=p, tol=t, C=a, class_weight=c_wght, solver=s)
                        pipeline = make_pipeline(clf)
                        pipeline.fit(X_train, y_train)
                        y_pred_trn = pipeline.predict(X_train)
                        trn_score = f1_score(y_train, y_pred_trn, average='weighted')
                        y_pred_val = pipeline.predict(X_val)
                        val_score = f1_score(y_val, y_pred_val, average='weighted')
                        print(f'pen: {p}, tol: {t}, C: {a}, class weight: {c_wght}, solver: {s}, train score: {trn_score:.2f}, val score: {val_score:.2f}')
                else:
                    clf = LogisticRegression(penalty=p, tol=t, C=a, class_weight=c_wght, solver='saga')
                    pipeline = make_pipeline(clf)
                    pipeline.fit(X_train, y_train)
                    y_pred_trn = pipeline.predict(X_train)
                    trn_score = f1_score(y_train, y_pred_trn, average='weighted')
                    y_pred_val = pipeline.predict(X_val)
                    val_score = f1_score(y_val, y_pred_val, average='weighted')
                    print(f'pen: {p}, tol: {t}, C: {a}, class weight: {c_wght}, solver: {s}, train score: {trn_score:.2f}, val score: {val_score:.2f}')
                    

pen: l1, tol: 0.0025, C: 0.01, class weight: balanced, solver: sag, train score: 0.82, val score: 0.83
pen: l2, tol: 0.0025, C: 0.01, class weight: balanced, solver: sag, train score: 0.82, val score: 0.82
pen: l2, tol: 0.0025, C: 0.01, class weight: balanced, solver: saga, train score: 0.82, val score: 0.82
pen: l1, tol: 0.005, C: 0.01, class weight: balanced, solver: saga, train score: 0.82, val score: 0.83
pen: l2, tol: 0.005, C: 0.01, class weight: balanced, solver: sag, train score: 0.82, val score: 0.82
pen: l2, tol: 0.005, C: 0.01, class weight: balanced, solver: saga, train score: 0.82, val score: 0.82
pen: l1, tol: 0.0075, C: 0.01, class weight: balanced, solver: saga, train score: 0.82, val score: 0.83
pen: l2, tol: 0.0075, C: 0.01, class weight: balanced, solver: sag, train score: 0.82, val score: 0.82
pen: l2, tol: 0.0075, C: 0.01, class weight: balanced, solver: saga, train score: 0.82, val score: 0.82


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


pen: l1, tol: 0.001, C: 0.01, class weight: balanced, solver: saga, train score: 0.82, val score: 0.83
pen: l2, tol: 0.001, C: 0.01, class weight: balanced, solver: sag, train score: 0.82, val score: 0.82
pen: l2, tol: 0.001, C: 0.01, class weight: balanced, solver: saga, train score: 0.82, val score: 0.82
pen: l1, tol: 1.2, C: 0.01, class weight: balanced, solver: saga, train score: 0.74, val score: 0.75
pen: l2, tol: 1.2, C: 0.01, class weight: balanced, solver: sag, train score: 0.85, val score: 0.86
pen: l2, tol: 1.2, C: 0.01, class weight: balanced, solver: saga, train score: 0.78, val score: 0.79
pen: l1, tol: 1.25, C: 0.01, class weight: balanced, solver: saga, train score: 0.84, val score: 0.85
pen: l2, tol: 1.25, C: 0.01, class weight: balanced, solver: sag, train score: 0.82, val score: 0.83
pen: l2, tol: 1.25, C: 0.01, class weight: balanced, solver: saga, train score: 0.85, val score: 0.85
pen: l1, tol: 1.3, C: 0.01, class weight: balanced, solver: saga, train score: 0.81, 

pen: l1, tol: 1.4, C: 0.05, class weight: balanced, solver: saga, train score: 0.79, val score: 0.80
pen: l2, tol: 1.4, C: 0.05, class weight: balanced, solver: sag, train score: 0.79, val score: 0.80
pen: l2, tol: 1.4, C: 0.05, class weight: balanced, solver: saga, train score: 0.82, val score: 0.82
pen: l1, tol: 1.45, C: 0.05, class weight: balanced, solver: saga, train score: 0.83, val score: 0.83
pen: l2, tol: 1.45, C: 0.05, class weight: balanced, solver: sag, train score: 0.80, val score: 0.81
pen: l2, tol: 1.45, C: 0.05, class weight: balanced, solver: saga, train score: 0.82, val score: 0.82
pen: l1, tol: 1.5, C: 0.05, class weight: balanced, solver: saga, train score: 0.81, val score: 0.82
pen: l2, tol: 1.5, C: 0.05, class weight: balanced, solver: sag, train score: 0.81, val score: 0.81
pen: l2, tol: 1.5, C: 0.05, class weight: balanced, solver: saga, train score: 0.79, val score: 0.79
pen: l1, tol: 0.0025, C: 0.075, class weight: balanced, solver: saga, train score: 0.83, va

/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


...

pen: l1, tol: 0.001, C: 0.9, class weight: None, solver: saga, train score: 0.87, val score: 0.87


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


pen: l2, tol: 0.001, C: 0.9, class weight: None, solver: sag, train score: 0.87, val score: 0.87


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


pen: l2, tol: 0.001, C: 0.9, class weight: None, solver: saga, train score: 0.87, val score: 0.87
pen: l1, tol: 1.2, C: 0.9, class weight: None, solver: saga, train score: 0.86, val score: 0.86
pen: l2, tol: 1.2, C: 0.9, class weight: None, solver: sag, train score: 0.87, val score: 0.87
pen: l2, tol: 1.2, C: 0.9, class weight: None, solver: saga, train score: 0.87, val score: 0.87
pen: l1, tol: 1.25, C: 0.9, class weight: None, solver: saga, train score: 0.87, val score: 0.87
pen: l2, tol: 1.25, C: 0.9, class weight: None, solver: sag, train score: 0.87, val score: 0.86
pen: l2, tol: 1.25, C: 0.9, class weight: None, solver: saga, train score: 0.87, val score: 0.88
pen: l1, tol: 1.3, C: 0.9, class weight: None, solver: saga, train score: 0.87, val score: 0.87
pen: l2, tol: 1.3, C: 0.9, class weight: None, solver: sag, train score: 0.88, val score: 0.88
pen: l2, tol: 1.3, C: 0.9, class weight: None, solver: saga, train score: 0.87, val score: 0.88
pen: l1, tol: 1.35, C: 0.9, class weig

Top Logistic Models:
* pen: l2, tol: 1.4, C: 0.05, class weight: None, solver: sag, train score: 0.88, val score: 0.88
* pen: l2, tol: 1.3, C: 0.05, class weight: None, solver: sag, train score: 0.88, val score: 0.88
* pen: l2, tol: 1.25, C: 0.075, class weight: None, solver: sag, train score: 0.88, val score: 0.88
* pen: l2, tol: 1.35, C: 0.7, class weight: None, solver: sag, train score: 0.88, val score: 0.88
* pen: l2, tol: 1.25, C: 0.8, class weight: None, solver: sag, train score: 0.88, val score: 0.88
* pen: l2, tol: 1.35, C: 0.85, class weight: None, solver: sag, train score: 0.88, val score: 0.88
* pen: l2, tol: 1.3, C: 0.9, class weight: None, solver: sag, train score: 0.88, val score: 0.88


### Exhaustive Search (Super Fine)

In [39]:
penalty = ['l2']
class_wght = [None]
solver = ['sag']
alphas = [0.9, 0.95, 1.0, 1.01, 1.1, 1.5]
tols = [1.25, 1.30, 1.35, 1.4]
for c_wght in class_wght:
    for a in alphas:
        for t in tols:
            for p in penalty:
                for s in solver:
                    clf = LogisticRegression(penalty=p, tol=t, C=a, class_weight=c_wght, solver=s)
                    pipeline = make_pipeline(clf)
                    pipeline.fit(X_train, y_train)
                    y_pred_trn = pipeline.predict(X_train)
                    trn_score = f1_score(y_train, y_pred_trn, average='weighted')
                    y_pred_val = pipeline.predict(X_val)
                    val_score = f1_score(y_val, y_pred_val, average='weighted')
                    print(f'pen: {p}, tol: {t}, C: {a}, class weight: {c_wght}, solver: {s}, train score: {trn_score:.2f}, val score: {val_score:.2f}')

pen: l2, tol: 1.25, C: 0.9, class weight: None, solver: sag, train score: 0.87, val score: 0.87
pen: l2, tol: 1.3, C: 0.9, class weight: None, solver: sag, train score: 0.87, val score: 0.88
pen: l2, tol: 1.35, C: 0.9, class weight: None, solver: sag, train score: 0.88, val score: 0.88
pen: l2, tol: 1.4, C: 0.9, class weight: None, solver: sag, train score: 0.88, val score: 0.88
pen: l2, tol: 1.25, C: 0.95, class weight: None, solver: sag, train score: 0.87, val score: 0.87
pen: l2, tol: 1.3, C: 0.95, class weight: None, solver: sag, train score: 0.88, val score: 0.88
pen: l2, tol: 1.35, C: 0.95, class weight: None, solver: sag, train score: 0.86, val score: 0.86
pen: l2, tol: 1.4, C: 0.95, class weight: None, solver: sag, train score: 0.87, val score: 0.87
pen: l2, tol: 1.25, C: 1.0, class weight: None, solver: sag, train score: 0.87, val score: 0.87
pen: l2, tol: 1.3, C: 1.0, class weight: None, solver: sag, train score: 0.88, val score: 0.88
pen: l2, tol: 1.35, C: 1.0, class weight:

## Hyperparameter Tuning - Random Forest

In [44]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4, 8],
 'min_samples_split': [2, 5, 10, 15],
 'n_estimators': [10, 64, 118, 173, 227, 282, 336, 391, 445, 500]}


In [50]:
from sklearn.metrics import make_scorer
f1_scorer = make_scorer(f1_score, average='weighted')

In [51]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, 
                               scoring=f1_scorer, n_jobs = -1)

In [52]:
pipeline = make_pipeline(rf_random)
pipeline.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 33.5min finished


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [55]:
pipeline._final_estimator

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [57]:
clf = pipeline['classifier']

In [58]:
clf.best_params_

{'n_estimators': 500,
 'min_samples_split': 15,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'bootstrap': True}

In [62]:
clf = RandomForestClassifier(n_estimators=500, min_samples_split=15,
                             min_samples_leaf=1, max_features='sqrt',
                             bootstrap=True)
pipeline = make_pipeline(clf)

In [63]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [64]:
y_preds = pipeline.predict(X_val)
f1_score(y_preds, y_val, average='weighted')

0.9210238138349669

In [65]:
y_preds = pipeline.predict(X_test)

In [68]:
X_test = pd.concat([X_test, X_val], axis=0)
y_test = pd.concat([y_test, y_val], axis=0)

In [69]:
y_preds = pipeline.predict(X_test)

In [70]:
f1_score(y_preds, y_test, average='weighted')

0.9205109493617953

In [71]:
confusion_matrix(y_test, y_preds)

array([[6825,  105],
       [ 702,  194]])

In [72]:
pipeline.predict_proba(X_test)

array([[0.97849449, 0.02150551],
       [0.95942338, 0.04057662],
       [0.96097802, 0.03902198],
       ...,
       [0.90966072, 0.09033928],
       [0.96789057, 0.03210943],
       [0.9784553 , 0.0215447 ]])